In [19]:
from symfit import variables, Parameter, ODEModel, D, Fit, parameters
import numpy as np
import matplotlib.pyplot as plt
from time import time
from scipy.optimize import curve_fit
from scipy.integrate import odeint
from symfit.contrib.interactive_guess import InteractiveGuess2D

from time import time as start_time

%matplotlib tk

In [20]:
file = 'kineticfile.txt'
data = np.genfromtxt(file)
time = data[:, 0][1:]/1000
cd = data[:, 1][1:]
cdHe = data[:, 3][1:]
cdHe2 = data[:, 4][1:]
cdHe3 = data[:, 5][1:]
cdHe4 = data[:, 6][1:]
He = 1.768e14

total = cd+cdHe+cdHe2+cdHe3+cdHe4


In [21]:
#plt.style.use()

fig, ax = plt.subplots(figsize = (14, 5), dpi = 100)

ax.plot(time, total,'k', label = 'Total')
ax.plot(time, cd, lw=1, label = 'CD+')
ax.plot(time, cdHe, lw=1, label = 'CDHe+')
ax.plot(time, cdHe2, lw=1, label = 'CDHe2+')
ax.plot(time, cdHe3, lw=1, label = 'CDHe3+')
ax.plot(time, cdHe4, lw=1, label = 'CDHe4+')

ax.set_xlim(left = 0)
ax.set_yscale('log')
ax.legend()
ax.grid()


plt.show()
#plt.close()

In [22]:
plt.style.available

['bmh',
 'classic',
 'dark_background',
 'fast',
 'fivethirtyeight',
 'ggplot',
 'grayscale',
 'seaborn-bright',
 'seaborn-colorblind',
 'seaborn-dark-palette',
 'seaborn-dark',
 'seaborn-darkgrid',
 'seaborn-deep',
 'seaborn-muted',
 'seaborn-notebook',
 'seaborn-paper',
 'seaborn-pastel',
 'seaborn-poster',
 'seaborn-talk',
 'seaborn-ticks',
 'seaborn-white',
 'seaborn-whitegrid',
 'seaborn',
 'Solarize_Light2',
 'tableau-colorblind10',
 '_classic_test',
 'ieee',
 'no-latex',
 'science',
 'subfigure']

# Rate Equations

$$\frac{d(CD^+)}{dt} = -[(k31) * CD^+ * He^2] + [(kCID1) * CDHe^+ * He] $$ 
$$\frac{d(CDHe^+)}{dt} = -[(k32) * CDHe^+ * He^2] + [(kCID2) * CDHe2^+ * He] + [(k31) * CD^+ * He^2] - [(kCID1) * CDHe^+ * He] $$ 
$$\frac{d(CDHe2^+)}{dt} = -[(k33) * CDHe2^+ * He^2] + [(kCID3) * CDHe3^+ * He] + [(k32) * CDHe^+ * He^2] - [(kCID2) * CDHe2^+ * He] $$
$$\frac{d(CDHe3^+)}{dt} = -[(k34) * CDHe3^+ * He^2] + [(kCID4) * CDHe4^+ * He] + [(k33) * CDHe2^+ * He^2] - [(kCID3) * CDHe3^+ * He] $$ 
$$\frac{d(CDHe4^+)}{dt} = [(k34) * CD^+ * He^2] - [(kCID4) * CDHe^+ * He] $$ 


### Rate Constants to fit K31, K32, K33, K34, KCID1, KCID2, KCID3, KCID4

In [23]:
CD, CDHe, CDHe2, CDHe3, CDHe4, t = variables('CD, CDHe, CDHe2, CDHe3, CDHe4, t')
#k31, k32, k33, k34, kCID1, kCID2, kCID3, kCID4 = parameters('k31, k32, k33, k34, kCID1, kCID2, kCID3, kCID4')

kmin, kcid_min = 1e-30, 1e-15
kmax, kcid_max = 1e-29, 1e-14

print(f'kmin: {kmin} kmax: {kmax}\nkcid_min: {kcid_min} kcid_max: {kcid_max}')

k31 = Parameter('k31', min=kmin, max=kmax)
k32 = Parameter('k32', min=kmin, max=kmax)
k33 = Parameter('k33', min=kmin, max=kmax)
k34 = Parameter('k34', min=kmin, max=kmax)

kCID1 = Parameter('kCID1', min=kcid_min, max=kcid_max)
kCID2 = Parameter('kCID2', min=kcid_min, max=kcid_max)
kCID3 = Parameter('kCID3', min=kcid_min, max=kcid_max)
kCID4 = Parameter('kCID4', min=kcid_min, max=kcid_max)


initial_cond = {
    t: 0,
    CD: cd[0],
    CDHe: 0,
    CDHe2: 0,
    CDHe3: 0,
    CDHe4: 0
}

r31 = k31*CD*He**2
rcid1 = kCID1*CDHe*He

r32 = k32*CDHe*He**2
rcid2 = kCID2*CDHe2*He

r33 = k33*CDHe2*He**2
rcid3 = kCID3*CDHe3*He

r34 = k34*CDHe3*He**2
rcid4 = kCID4*CDHe4*He

rate_model = {
    D(CD, t): -r31+rcid1,
    D(CDHe, t): -r32+rcid2+r31-rcid1,
    D(CDHe2, t): -r33+rcid3+r32-rcid2,
    D(CDHe3, t): -r34+rcid4+r33-rcid3,
    D(CDHe4, t): +r34-rcid4
}

ode_model = ODEModel(rate_model, initial=initial_cond)

# Generate some data
tdata = np.linspace(0, 3, 1000)

kmin: 1e-30 kmax: 1e-29
kcid_min: 1e-15 kcid_max: 1e-14


In [24]:
#guess = InteractiveGuess2D(ode_model, t=time, CD=cd, CDHe=cdHe, CDHe2=cdHe2, CDHe3=cdHe3, CDHe4=cdHe4)
#guess.execute()

In [25]:
# with Fit
t0 = start_time()

fit = Fit(ode_model, t=time, CD=cd, CDHe=cdHe, CDHe2=cdHe2, CDHe3=cdHe3, CDHe4=cdHe4)
fit_result = fit.execute()

t1 = start_time()
print(f'Time taken: {t1-t0} s')

CD, CDHe, CDHe2, CDHe3, CDHe4 = ode_model(t=tdata, **fit_result.params)

print(fit_result)
print(f'Parameters: {fit_result.params}' )

C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Repeated convergence failures (perhaps bad Jacobian or tolerances). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
C:\ProgramData\Anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


Time taken: 2.416018009185791 s

Parameter Value        Standard Deviation
k31       1.000000e-29 None
k32       1.000000e-29 None
k33       1.000000e-29 None
k34       1.000000e-29 None
kCID1     1.000000e-14 None
kCID2     1.000000e-14 None
kCID3     1.000000e-14 None
kCID4     1.000000e-14 None
Fitting status message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Number of iterations:   9
Regression Coefficient: -9.195065433669399

Parameters: OrderedDict([('k31', 1e-29), ('k32', 1e-29), ('k33', 1e-29), ('k34', 1e-29), ('kCID1', 1e-14), ('kCID2', 1e-14), ('kCID3', 1e-14), ('kCID4', 1e-14)])


In [26]:
print(f'Parameters: {fit_result.params}' )
print(fit_result)
fit_result.model

Parameters: OrderedDict([('k31', 1e-29), ('k32', 1e-29), ('k33', 1e-29), ('k34', 1e-29), ('kCID1', 1e-14), ('kCID2', 1e-14), ('kCID3', 1e-14), ('kCID4', 1e-14)])

Parameter Value        Standard Deviation
k31       1.000000e-29 None
k32       1.000000e-29 None
k33       1.000000e-29 None
k34       1.000000e-29 None
kCID1     1.000000e-14 None
kCID2     1.000000e-14 None
kCID3     1.000000e-14 None
kCID4     1.000000e-14 None
Fitting status message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
Number of iterations:   9
Regression Coefficient: -9.195065433669399



In [27]:
# Eval the normal way. (without fit)
CD, CDHe, CDHe2, CDHe3, CDHe4 = ode_model(t=tdata, k31=2.7e-30, k32=6e-30, k33=32e-30, k34=15e-28, kCID1=1.9e-15, kCID2=4e-15, kCID3=24e-15, kCID4=15e-14)


# with Fit
#ode_model = ODEModel(rate_model, initial=initial_cond)

#fit = Fit(ode_model, t=time, CD=cd, CDHe-cdHe, CDHe2=cdHe2, CDHe3=cdHe3, CDHe4=cdHe4)
#fit_result = fit.execute()

#CD, CDHe, CDHe2, CDHe3, CDHe4 = ode_model(t=tdata, **fit_result.params)

######################################################################################################################################################################

fig2, ax2 = plt.subplots(figsize = (14, 5), dpi = 100)

ax2.plot(tdata, CD, label = 'CD+')
ax2.plot(time, cd, '.', label = 'CD+_exp')

ax2.plot(tdata, CDHe, label = 'CDHe+')
ax2.plot(time, cdHe, '.', label = 'CDHe+_exp')

ax2.plot(tdata, CDHe2, label = 'CDHe2+')
ax2.plot(time, cdHe2, '.', label = 'CDHe2+_exp')

ax2.plot(tdata, CDHe3, label = 'CDHe3+')
ax2.plot(time, cdHe3, '.', label = 'CDHe3+_exp')

ax2.plot(tdata, CDHe4, label = 'CDHe4+')
ax2.plot(time, cdHe4, '.', label = 'CDHe4+_exp')

ax2.set_yscale('log')
ax2.set_xlabel('Time (s)')
ax2.set_ylabel('Counts')
ax2.set_title('Ternary Reactions rate')
ax2.legend()
ax2.grid()

plt.show()
#plt.close()